In [1]:
import numpy as np
import pandas as pd
import xarray as xr
from joblib import dump, load
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import glob

import matplotlib as mpl

import matplotlib.pyplot as plt

import math as mt

import cartopy.crs as ccrs

import cartopy.feature as cfeature

from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter

#AO Calculation

In [2]:
# Calculate AO - input std normal latitude wighted 1000mb z
ZData = xr.open_dataset('../../../Data/CMIP6/MRI-ESM2-0/processed/weighted1000Std.nc')
#SLPData = xr.open_dataset('../eraDown/ERA5_2degree_Down/DailyMean/weighted1000.nc')

ZData.zg.values
ZData.zg.values=xr.where(np.isnan(ZData.zg.values),  0.000000000001,ZData.zg.values)
ZData.zg.values

array([[[1.0000000e-12, 1.0000000e-12, 1.0000000e-12, ...,
         1.0000000e-12, 1.0000000e-12, 1.0000000e-12],
        [1.0000000e-12, 1.0000000e-12, 1.0000000e-12, ...,
         1.0000000e-12, 1.0000000e-12, 1.0000000e-12],
        [1.0000000e-12, 1.0000000e-12, 1.0000000e-12, ...,
         1.0000000e-12, 1.0000000e-12, 1.0000000e-12],
        ...,
        [5.7029354e+01, 5.7059544e+01, 5.7088417e+01, ...,
         5.6941372e+01, 5.6976742e+01, 5.7005386e+01],
        [4.6716553e+01, 4.6717819e+01, 4.6713989e+01, ...,
         4.6720196e+01, 4.6716892e+01, 4.6717125e+01],
        [3.1329012e+01, 3.1325508e+01, 3.1323137e+01, ...,
         3.1345144e+01, 3.1342911e+01, 3.1337147e+01]],

       [[1.0000000e-12, 1.0000000e-12, 1.0000000e-12, ...,
         1.0000000e-12, 1.0000000e-12, 1.0000000e-12],
        [1.0000000e-12, 1.0000000e-12, 1.0000000e-12, ...,
         1.0000000e-12, 1.0000000e-12, 1.0000000e-12],
        [1.0000000e-12, 1.0000000e-12, 1.0000000e-12, ...,
         1.000

In [3]:


t1=ZData.zg.stack(z=("lat", "lon"))
# fit scaler on training data
zg_PC =PCA().fit_transform(t1)


In [4]:
print(zg_PC[:,0].shape)
AO_ds = xr.Dataset({'AO': (('time'), zg_PC[:,0])}, coords={'time': ZData.time})


norm = StandardScaler().fit(zg_PC[:,0].reshape(-1, 1))
    # transform training data
aoX = norm.transform(zg_PC[:,0].reshape(-1, 1))
aoX.shape

AO_ds = xr.Dataset({'AO': (('time'), aoX[:,0]*-1.0)}, coords={'time': ZData.time})


(4810,)


In [5]:
AO_ds.to_netcdf('MRI-ESM2-0-AOindex-NDJF-Daily-1980-2014.nc')

In [6]:
AO_ds.AO.where(AO_ds.AO==AO_ds.AO.max(), drop=True).squeeze()

<xarray.DataArray 'AO' ()>
array(2.5997612, dtype=float32)
Coordinates:
    time     datetime64[ns] 2005-11-04T12:00:00

In [7]:
#AO_ds.AO.where(AO_ds.AO>2.0, drop=True).squeeze()

In [8]:
AO_ds.AO.where(AO_ds.AO==AO_ds.AO.min(), drop=True).squeeze()

<xarray.DataArray 'AO' ()>
array(-3.4750159, dtype=float32)
Coordinates:
    time     datetime64[ns] 1986-01-24T12:00:00

In [9]:
#AO_ds.AO.where(AO_ds.AO<-2, drop=True).squeeze()

#EU Calculation

In [10]:
zLevData = xr.open_dataset('../../../Data/CMIP6/MRI-ESM2-0/processed/NDJF_New_zg_day_MRI-ESM2-0_historical_r1i1p1f1_gn_18500101-20141231.nc')

In [11]:
zLevData.plev

<xarray.DataArray 'plev' (plev: 8)>
array([100000.,  85000.,  70000.,  50000.,  25000.,  10000.,   5000.,   1000.])
Coordinates:
  * plev     (plev) float64 1e+05 8.5e+04 7e+04 5e+04 2.5e+04 1e+04 5e+03 1e+03
Attributes:
    standard_name:  air_pressure
    long_name:      pressure
    units:          Pa
    positive:       down
    axis:           Z

In [12]:
z500=zLevData.zg.sel(plev=50000.0,method='nearest')
z500

<xarray.DataArray 'zg' (time: 4810, lat: 160, lon: 320)>
[246272000 values with dtype=float32]
Coordinates:
  * time     (time) datetime64[ns] 1975-01-01T12:00:00 ... 2014-12-31T12:00:00
  * lon      (lon) float64 0.0 1.125 2.25 3.375 4.5 ... 355.5 356.6 357.8 358.9
  * lat      (lat) float64 -89.14 -88.03 -86.91 -85.79 ... 86.91 88.03 89.14
    plev     float64 5e+04
Attributes:
    standard_name:  geopotential_height
    long_name:      Geopotential Height
    units:          m
    comment:        Geopotential is the sum of the specific gravitational pot...
    original_name:  GH
    cell_methods:   time: mean
    cell_measures:  area: areacella
    history:        2019-05-10T18:46:31Z altered by CMOR: replaced missing va...

In [13]:
z145E40N=z500.sel(lat=40,lon=145,method='nearest')

In [14]:
#z145E40N

In [15]:
z20E55N=z500.sel(lat=55,lon=20,method='nearest')

In [16]:
#z20E55N

In [17]:
z75E55N=z500.sel(lat=55,lon=75,method='nearest')

In [18]:
#z75E55N

In [19]:
#Calculate Anomaly
z145E40N_Anom = z145E40N.groupby("time.dayofyear") - z145E40N.groupby("time.dayofyear").mean("time")
z75E55N_Anom  = z75E55N.groupby("time.dayofyear")  - z75E55N.groupby("time.dayofyear").mean("time")
z20E55N_Anom  = z20E55N.groupby("time.dayofyear")  - z20E55N.groupby("time.dayofyear").mean("time")

In [20]:
# calculate std normal anomalies
z145E40N_AnomStd = z145E40N_Anom.groupby("time.dayofyear")  / z145E40N.groupby("time.dayofyear").std("time")
z75E55N_AnomStd  = z75E55N_Anom.groupby("time.dayofyear")   / z75E55N.groupby("time.dayofyear").std("time")
z20E55N_AnomStd  = z20E55N_Anom.groupby("time.dayofyear")   / z20E55N.groupby("time.dayofyear").std("time")

In [21]:
EUVal = -(1.0/4.0)*z20E55N_AnomStd + (1.0/2.0)*z75E55N_AnomStd - (1.0/4.0)*z145E40N_AnomStd

In [22]:
EUVal

<xarray.DataArray 'zg' (time: 4810)>
array([-1.45876938, -0.88760952, -1.25634349, ...,  0.18818272,
        0.14539498,  0.02621327])
Coordinates:
  * time       (time) datetime64[ns] 1975-01-01T12:00:00 ... 2014-12-31T12:00:00
    plev       float64 5e+04
    dayofyear  (time) int64 1 2 3 4 5 6 7 8 ... 358 359 360 361 362 363 364 365
    lon        float64 145.1

In [23]:
EU_ds = xr.Dataset({'EU': (('time'), EUVal)}, coords={'time': zLevData.time})

In [24]:
EU_ds.to_netcdf('MRI-ESM2-0-EUindex-NDJF-Daily-1980-2014.nc')